In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [4]:
def draw_landmarks(image,results):
    """mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                                                                                         mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    
    #mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                                                                                        mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))"""
    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                                                                                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                                                                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [5]:
"""cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
else:
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic: 
        while cap.isOpened():
            ret,frame = cap.read()

            image,results = mediapipe_detection(frame,holistic)
            print(results)
            
            draw_landmarks(image,results)

            cv2.imshow('OpenCV feed',image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()"""

'cap = cv2.VideoCapture(0)\nif not cap.isOpened():\n    print("Error: Could not open video device.")\nelse:\n    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic: \n        while cap.isOpened():\n            ret,frame = cap.read()\n\n            image,results = mediapipe_detection(frame,holistic)\n            print(results)\n            \n            draw_landmarks(image,results)\n\n            cv2.imshow(\'OpenCV feed\',image)\n            if cv2.waitKey(10) & 0xFF == ord(\'q\'):\n                break\n        cap.release()\n        cv2.destroyAllWindows()'

In [5]:
def extract_keypoints(results):
    #pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])

In [6]:
def list_folders_in_directory():
    directory_path = r"D:\SpectoV\Hand_Gesture_Recognition\data\level_1_data\ASL_self_created_data"
    try:
        items = os.listdir(directory_path)
        folders = [item for item in items if os.path.isdir(os.path.join(directory_path, item))]
        return folders,directory_path
    except FileNotFoundError:
        print(f"The directory {directory_path} does not exist.")
        return [],directory_path
    except Exception as e:
        print(f"An error occurred: {e}")
        return [],directory_path

In [7]:
def list_video_files(folder_path):
    video_files = []
    for file in os.listdir(folder_path):
        if file.endswith((".mp4", ".avi", ".mkv", ".mov", ".wmv")):
            video_files.append(file)
    return video_files

In [8]:
def get_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    key_points = []
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}.")
        return
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)

            keypoints = extract_keypoints(results)
            key_points.append(keypoints)
            
            cv2.imshow('Video Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    
    return np.array(key_points)

In [9]:
def data():
    key_point_data_path = r"D:\SpectoV\Hand_Gesture_Recognition\research\level_3\key_points"
    actions, directory_path = list_folders_in_directory()
    num_videos = 40
    fps = 30
    duration = 2
    frame_count = fps * duration
    for action in tqdm(actions, desc="Processing Actions"):
        path = os.path.join(directory_path, str(action))
        videos = list_video_files(path)
        for video in tqdm(videos, desc=f"Processing Videos in {action}", leave=False):
            try:
                action_path = os.path.join(key_point_data_path, action, str(video).split(".")[0])
                os.makedirs(action_path, exist_ok=True) 
                key_points = get_keypoints(os.path.join(path, video))
                for i,point in zip(range(1,len(key_points)+1),key_points):
                    file_path = os.path.join(action_path,str(i))
                    np.save(file_path, point)
            except Exception as e:
                print(f"Error processing {video}: {e}")

In [10]:
data()

Processing Actions: 100%|██████████| 2/2 [12:15<00:00, 367.74s/it]
